# Displaying a basic map

In [1]:
# Lines that start with `#` are comments, they don't execute code, these next lines import the necessary libraries to process the data
import os
import re
import folium
import openpyxl
import pandas as pd
import earthpy as et
import geopandas as gpd
from pathlib import Path

In [2]:
# Create a map using the City of Columbus's GPS Coordinates
map_osm = folium.Map(location=[39.983334, -82.983330], zoom_start=11)
map_osm

# Building out a map from datapoints

List of relevant Bridges to D6 - Within "D6 Bridge Records project (ODOT internal)": `Bridge_Record_CSVs_(Tracked_Changes)/BridgeListFromAssetWise.xlsx`

Excel file is every bridge in Assetwise assigned to D6, filtered in assetwise and then exported as an excel file.

Loading file via python (change the path in the next cell to use your own data):

In [3]:
assetwise_export_path = Path("C:\\Users\\dparks1\\PycharmProjects\\D6_Bridge_Records\\Bridge_Record_CSVs_(Tracked_Changes)\\BridgeListFromAssetWise.xlsx")

In [4]:
wb = openpyxl.load_workbook(assetwise_export_path)

make sure the correct file was found and that the hyperlinks are working/available by printing out first hyperlink:

In [5]:
ws = wb["BridgeListFromAssetWise"]
print(ws.cell(row=2, column=1).hyperlink.target)

https://ohiodot-it.bentley.com/bridgedetail.aspx?type=0&as_id=78769


## Using Pandas to access and filter raw data

openpyxl is good for accessing aspects of excel's interface, like reading hyperlinks, but to filter/modify data, it needs to be loaded into pandas

filter bridges by state maintenance (Column H - NBI 21 = 01)

In [6]:
# Read the data into memory
all_bridges_df = pd.read_excel(assetwise_export_path)

# Filter the data
d6_bridges = all_bridges_df.loc[all_bridges_df['NBI 021: Maintenance Responsibility(Report)'] == 1.0]

Make sure the only values left in the table are one's ODOT Maintains

In [7]:
# Displays all unique values contained in the specified column of the dataframe
d6_bridges['NBI 021: Maintenance Responsibility(Report)'].unique()

array([1.])

## Getting SFN from larger string

The format assetwise uses to hold the bridge name is regular, but not simple. Although Regex isn't necessarily required for this bit, as the built in split function for strings would work, it's powerful and understanding it is helpful.

The goal is to get all the characters between the `(` and `)` from the following value. [This website](https://regex101.com/) is useful for testing the patterns, in this case, the pattern used is `'\((.*?)\)'`.

In [8]:
# Display the text that will be searched in the next cell
d6_bridges.iloc[0]['Asset Name']

'FAY-00035-2066 _(2400588)'

In [9]:
# Use a regular expression to extract the necessary value from the data and print it
bridge_sfn = re.search(r'\((.*?)\)',d6_bridges.iloc[0]['Asset Name']).group(1)
bridge_sfn

'2400588'

Now that we have the bridge's SFN, we can use the existing Civilpy tools to get a map for the structure, and various attributes from TIMs for it.

In [10]:
# Load the library I wrote/published
from civilpy.state.ohio.dot import TimsBridge

# Lookup bridge values by sfn
bridge_lookup_result = TimsBridge(bridge_sfn)


TIMS Bridge Initiated


Retrieving data from url at https://gis.dot.state.oh.us//arcgis/rest/services/TIMS/Assets/MapServer/5/27715386?f=pjson



In [12]:
bridge_lookup_result.map